# Word Embedding y Word2Vec

### Motivación: representar las dimensiones semánticas de cada palabra

1. I want an orange.
2. I wantrepresentar an apple.

- Los enfoques <i>bag of words</i> no tienen la capacidad de calcular que las frases 1 y 2 son muy similares porque no tienen una manera de representar que las palabras 'orange' y 'apple' comparten caracterícas (<i>features</i>) comunes.

Los enfoques ingenuos tieden a representar las palabras como vectores "1-Hot". Por ejemplo, supongamos que tenemos un vocabulario de sólo cinco palabras: King, Queen, Man, Woman y Child. Se codificaría la palabra 'Queen' como:

<img src="img/word2vec1.png"/>

- Sería más interesante poder representar la semántica de cada palabra tomando en cuentas ciertas características. 

<img src="img/word2vec2.png"/>


### Definición

El concepto de **word embedding** se refiere a un conjunto de técnicas utilizadas para aprender representaciones matemáticas, tipicamente vectores, de cada palabra.

Una de las técnicas más populares es __Word2Vec__ propuesto por un equipo de investigación de Google en 2013 (Efficient Estimation of Word Representations in Vector Space [Mikolov et al., 2013]).

Alternativas populares son __GloVe__ (propuesta por la Universidad de Stanford en 2014) y __FastText__ (propuesta por Facebook en 2016), que extende Word2Vec para considerar de mejor manera las palabras con errores ortográficas.

### Algunas propiedades de los word embeddings

- Tener representaciones vectoriales de las palabras permite calcular "razonamiento" de tipo __King - Man + Woman = ?__ y llegar a un resultado cerca de __Queen__.

<img src="img/word2vec4.png"/>

- Tener representaciones vectoriales de las palabras permite realizar razonamientos analógicos de tipo __A es a B, lo que C es a ..__ . Este tipo de propiedades es muy útil para aplicaciones de _Question Answering_ por ejemplo. Las respuestas a las pregutas siguientes <i>¿Cuál es la capital de Chile?</i> o <i>¿Cuáles son los clubs de fútbol en Chile?</i> se pueden responder adicionando vectores.

<img src="img/word2vec6.png"/>

<img src="img/word2vec7.png"/>

<img src="img/word2vec8.png"/>

### ¿Cómo se aprenden los vectores? - Redes neuronales

ara construir sus vectores, Word2Vec utiliza un dataset de entrenamiento y algoritmos de aprendizaje basados en redes neuronales (__Continuous Bag of Words__ (CBOW), o modelo __Skip Gram__). El objetivo de esta fase de aprendizaje es aprender cuáles son las palabras _X_ más probables de aparecer en el contexto de una palabra _y_.

<img src="img/word2vec5.png"/>

Por ejemplo, ¿cuál es la probabilidad de tener la palabra 'perro' si aparece la palabra 'pelota' en el contexto?

<code>Los expertos explican que los __perros__ persiguen __pelotas__ en movimiento como parte de un comportamiento instintivo. Aunque no todos los perros tienen tan despiertos su instinto de caza, esto no impide que la mayoría de ellos sí disfruten, y mucho, de los juegos que incluyen persecuciones de una saltarina __pelota__ que bota delante de ellos. </code>

__Algoritmo CBOW__

Las palabras de contexto forman la capa de entrada. Si el tamaño del vocabulario es V, estos serán vectores de dimensión V con sólo uno de los elementos establecido en uno, y el resto todos los ceros. Hay una sola capa oculta y una capa de salida.

<img src="img/word2vec9.png"/>

### Un ejemplo práctico

La clase <code>word2vec</code> de Gensim permite word embeddings de palabras (ver documentación: https://radimrehurek.com/gensim/models/word2vec.html).

Esta clase tiene varios parametros, en particular:
- <code>sentences</code>: una lista de palabras o de frases que sirve para entrenar el modelo
- <code>sg</code>: define que algoritmos de aprendizaje utilizar (0=CBOW, 1=skip-gram)
- <code>vector_size</code>: define la dimensión de los vectores que se desea extraer
- <code>window</code>: define el número de palabras considerar a la izquierda y a la derecha de una palabra
- <code>min_count</code>: ignorar las palabras que aparecen menos de _min_count_
y otros asociados a la parametrización de la fase de aprendizaje de la red neuronal (que no detallaremos en esta parte del curso):
- <code>alpha</code>: el _learning rate_ utilizado para optimizar los parametros de la red neuronal.
- <code>iter</code>: número de iteraciones (epocas) sobre el dataset para encontrar los parametreos que optimizan la red neuronal.

In [27]:
from gensim.models import word2vec

Para entrenar nuestro modelo Word2Vec, podemos utilizar nuestros propios datasets o utilizar datasets genericos existentes. Para empezar, utilizaremos 100 MB de textos extraidos de Wikipedia en inglés, para generar vectores de 200 dimensiones.

In [28]:
sentences = word2vec.Text8Corpus('../datasets/text8.txt')

In [29]:
model = word2vec.Word2Vec(sentences,vector_size=200,hs=1)
#model=word2vec.Word2Vec.load("text8_model")

In [30]:
print(model)

Word2Vec(vocab=71290, vector_size=200, alpha=0.025)


Ahora que hemos aprendido nuestro modelo, tratemos de resolver la ecuación <code>King - Man + Woman</code>.

En otras palabras buscamos cuál es el vector más similar al vector que adiciona positivamente 'King' y 'Woman' y negativamente 'Man'.

In [31]:
model.wv.most_similar(positive=['woman','king'],negative=['man'],topn=5)

[('queen', 0.5515807867050171),
 ('throne', 0.4885711073875427),
 ('regent', 0.487594872713089),
 ('monarch', 0.479781836271286),
 ('prince', 0.4787699282169342)]

In [32]:
model.wv.most_similar(positive=["conflict"])

[('conflicts', 0.6832011342048645),
 ('clashes', 0.667018473148346),
 ('hostilities', 0.6493234634399414),
 ('tensions', 0.6465848684310913),
 ('confrontation', 0.6254630088806152),
 ('struggle', 0.6226808428764343),
 ('confrontations', 0.6206727623939514),
 ('strife', 0.6187384724617004),
 ('disagreements', 0.6130479574203491),
 ('dispute', 0.5851774215698242)]

In [33]:
model.wv.most_similar(positive=["conflict","weapon"])

[('confrontation', 0.5802947282791138),
 ('weapons', 0.5498438477516174),
 ('warfare', 0.5303004384040833),
 ('rifle', 0.5301260948181152),
 ('struggle', 0.5106883645057678),
 ('confrontations', 0.5069210529327393),
 ('pistol', 0.5058635473251343),
 ('fighting', 0.4914857745170593),
 ('ammunition', 0.48768478631973267),
 ('conflicts', 0.48271921277046204)]

In [34]:
model.wv.most_similar(positive=["conflict"],negative=["weapon"])

[('disagreements', 0.49566638469696045),
 ('clashes', 0.4933446943759918),
 ('conflicts', 0.4849163293838501),
 ('disputes', 0.45843130350112915),
 ('tensions', 0.4419403374195099),
 ('hostilities', 0.4400283694267273),
 ('dispute', 0.4245198667049408),
 ('strife', 0.4076556861400604),
 ('infighting', 0.3949545621871948),
 ('antagonism', 0.39344874024391174)]

In [35]:
model.wv.most_similar(positive=["life"])

[('childhood', 0.5314176678657532),
 ('career', 0.5136758685112),
 ('lives', 0.5071121454238892),
 ('experiences', 0.4583725035190582),
 ('humanity', 0.4511985182762146),
 ('work', 0.4490172266960144),
 ('mankind', 0.43727898597717285),
 ('universe', 0.4271804094314575),
 ('teens', 0.4086006283760071),
 ('happiness', 0.40494653582572937)]

In [36]:
model.wv.most_similar(positive=["life"],negative=["money"])

[('childhood', 0.3892081081867218),
 ('adolescence', 0.3318144679069519),
 ('experiences', 0.330782413482666),
 ('spiritual', 0.32153668999671936),
 ('career', 0.3140907287597656),
 ('stillness', 0.3087413012981415),
 ('ueshiba', 0.30592378973960876),
 ('biography', 0.3013848662376404),
 ('reptilian', 0.29709941148757935),
 ('rebirth', 0.2940960228443146)]

Ver los parametros aprendidos por la red neuronal para una palabra dada:

In [37]:
model.wv['computer']

array([ 1.3508972 , -1.0290902 ,  0.97134167, -1.3990436 ,  0.37600255,
        0.68262666,  0.3371534 , -1.0618834 ,  0.7140528 ,  0.79375964,
       -1.580623  ,  0.08056355,  2.2957795 , -0.4423584 ,  0.85803866,
        1.6259893 ,  2.0807257 ,  0.27033195,  0.10869743, -0.7073341 ,
        0.93453103,  2.448685  , -1.065508  ,  0.6948831 , -0.3933674 ,
       -0.25398833,  0.0525438 ,  0.2760993 ,  0.03046661,  1.6357298 ,
       -0.68046504, -0.6552789 ,  0.18674538, -1.5934254 , -1.3352039 ,
       -0.11297392,  0.16831501, -3.093571  , -1.304632  , -0.36225113,
       -0.85680985, -0.6849744 , -0.6732246 ,  0.7346014 ,  0.04552502,
        0.9106011 , -0.8569324 , -0.78676486, -0.7548889 , -0.6239116 ,
       -0.10573357,  0.1871993 , -1.0517532 ,  1.2427821 ,  0.09027182,
        0.646295  ,  1.4707023 , -1.0996007 ,  0.71657985, -0.4825657 ,
        0.7736694 , -0.8214331 , -0.06812207,  0.21072268, -0.05821994,
        0.2834152 , -0.5038266 , -0.02220035, -0.7816633 , -0.42

Guardar el modelo:

In [15]:
model.save("text8_model")
model=word2vec.Word2Vec.load("text8_model")

In [16]:
model.wv.doesnt_match("breakfast cereal dinner lunch".split())

'cereal'

In [17]:
model.wv.doesnt_match("brazil chile france peru argentina".split())

'france'

In [18]:
model.wv.doesnt_match("apple pear banana hammer".split())

'hammer'

In [19]:
model.wv.similarity('man','woman')

0.73985076

In [20]:
model.wv.similarity('man','hammer')

0.31393042

In [21]:
model.wv.similarity('woman','hammer')

0.20366034

In [22]:
model.wv.similarity('man','engineer')

0.09999105

In [23]:
model.wv.similarity('woman','engineer')

0.036601752

In [24]:
model.wv.similarity('man','baby')

0.436531

In [25]:
model.wv.similarity('woman','baby')

0.56964356

### Cargar un modelo Word2Vec pre-entrenado para el español

ver: https://github.com/dccuchile/spanish-word-embeddings

In [41]:
from gensim.models import KeyedVectors

model = KeyedVectors.load_word2vec_format('./data/SBW-vectors-300-min5.bin.gz', binary=True) 

In [42]:
dog = model['perro']
print(dog.shape)
print(dog[:10])

(300,)
[ 0.1051706  -0.27460352 -0.21322592  0.261666    0.09946854 -0.02449877
  0.12955804 -0.34066245  0.3385692  -0.09923615]


In [48]:
print(model.similarity('mujer', 'ingeniería'))
print(model.similarity('hombre', 'ingenieria'))

0.12951772
0.15265214


In [49]:
print(model.similarity('mujer', 'bebe'))
print(model.similarity('hombre', 'bebe'))

0.4396093
0.40620732


In [53]:
print(model.similarity('chileno', 'violencia'))
print(model.similarity('venezolano', 'violencia'))
print(model.similarity('francés', 'violencia'))

0.1600553
0.20306623
0.13652363


In [54]:
print(model.similarity('chileno', 'criminal'))
print(model.similarity('venezolano', 'criminal'))
print(model.similarity('francés', 'criminal'))

0.17682078
0.19584653
0.10330092


In [57]:
model.most_similar(positive=["chileno"],topn=20)

[('chilena', 0.7700598239898682),
 ('Chile', 0.7495904564857483),
 ('peruano', 0.7162680625915527),
 ('chilenos', 0.7058151960372925),
 ('Chileno', 0.7000812292098999),
 ('argentino', 0.6901670694351196),
 ('boliviano', 0.668327808380127),
 ('Antofagasta', 0.6591013073921204),
 ('Talca', 0.6463863849639893),
 ('ecuatoriano', 0.6427898406982422),
 ('ariqueño', 0.6422776579856873),
 ('chilenas', 0.6416400671005249),
 ('Valparaíso', 0.6410984992980957),
 ('Pavez', 0.6397318243980408),
 ('penquista', 0.6381476521492004),
 ('Iturra', 0.6330534219741821),
 ('Iquique', 0.6310879588127136),
 ('nortino', 0.6273276209831238),
 ('iquiqueño', 0.6271899938583374),
 ('Temuco', 0.6224685311317444)]

### Limitaciones de los word embeddings

Las técnicas de Word Embeddings dan resultados muy interesantes pero tienen dos principales limitaciones:

1) No permiten tomar en cuenta el orden entre las palabras.

Ejemplo: "Estamos aqui para trabajar y no jugar" vs. "Estamos aqui para jugar y no trabajar"

2) No permiten tomar en cuenta que ciertas palabras cambian de significado según el contexto.

Ejemplo: "I lost my computer __mouse__"

Para mejorar estas limitaciones:

- Combinar Word Embedding con redes neuronales (convolucionales (CNN) o secuenciales (Transformers, BERT, GPT-3)) que toman en cuenta el orden entre las palabras y el contexto de las palabras.
